In [9]:
from path_arquivos import *
import pandas as pd
caminho = r'z:\1 - CD Dia\4 - Equipe PCL\6.1 - Inteligência Logística\6.1.1 Analise de Cortes\2025\Acompanhamento de produtos cortados.xlsx'

df_67 = pd.read_excel(caminho, sheet_name= 'CORTE 2025')
df_86 = pd.read_excel(ar_xls.ar_86, usecols= ['Código', 'Descrição ', 'Estoque', 'Qtde Pedida', 'Bloqueado(Qt.Bloq.-Qt.Avaria)', 'Qt.Avaria', 'Reservado', 'Disponível'])

display(df_67.columns)
display(df_86.columns)

Index(['DATA', 'CARR.', 'COD', 'DESCRIÇÃO', 'PEDIDO', 'QTDE ORIG', 'VL. ORIG',
       'RUA', 'PREDIO', 'APTO', 'ESTOQUE', 'QTDE CORTE', 'VL CORTE', 'HORA',
       'MIN', 'MOTIVO', 'COD FUNC', 'FUNCIONARIO', 'CONCAT', 'COUNT', 'IF',
       'MÊS', 'TURNO'],
      dtype='object')

Index(['Código', 'Descrição ', 'Estoque', 'Qtde Pedida',
       'Bloqueado(Qt.Bloq.-Qt.Avaria)', 'Qt.Avaria', 'Reservado',
       'Disponível'],
      dtype='object')

CORPO DO SCRIPTS

In [35]:
df_67 = df_67.loc[df_67['MÊS'] == 'setembro']
df_67['HORAS'] = pd.to_datetime(df_67['HORA'].astype(str) + ":" + df_67['MIN'].astype(str), format='%H:%M').dt.time

var_corte = df_67.groupby('COD').agg(
    QTDE_DIAS = ('DATA', 'nunique'),
    TOTAL_CORTE = ('VL CORTE', 'sum'),
    QTDE_CORTE = ('QTDE CORTE', 'sum')
).reset_index().sort_values(by=['QTDE_DIAS', 'QTDE_CORTE'], ascending= False)
df_86['BLOQUEADO'] = df_86['Bloqueado(Qt.Bloq.-Qt.Avaria)'].fillna(0).astype(int) + df_86['Qt.Avaria'].fillna(0).astype(int)
var_86 = df_86[['Código', 'Descrição ', 'Estoque', 'BLOQUEADO']].copy()
df = var_corte.merge(var_86, left_on= 'COD', right_on= 'Código', how= 'left').drop(columns='Código')
df['var'] = df['BLOQUEADO'] - df['Estoque']

validar = df.loc[(df['var'] < 0) & (df['QTDE_DIAS'] > 1)]


display(validar)

,COD,QTDE_DIAS,TOTAL_CORTE,QTDE_CORTE,Descrição,Estoque,BLOQUEADO,var
0,464141,3,5224.50,90.0,CREME LEITE TIROL 200GR,382.0,14.0,-368.0
2,468973,2,718.45,133.0,ACHOC NESCAU 200GR,233.0,3.0,-230.0
4,444365,2,1454.60,41.0,SUCO NUTRINECTAR PRONTO 200ML UVA,23.0,0.0,-23.0
8,450655,2,303.98,8.0,SAB LUX 125GR BUQUE DE JASMIM,590.0,0.0,-590.0
17,280000,2,182.79,2.0,LIMP VEJA 500ML M USO ORIGINAL,7.0,0.0,-7.0
18,428511,2,29.04,2.0,ESPONJA 3M LIMPEZA SC BRITE NAO RISCA,368.0,0.0,-368.0


In [40]:

comp = df_67.loc[df_67['COD'] == 428511]
comp = comp[['DATA','HORAS', 'COD', 'DESCRIÇÃO', 'MOTIVO', 'FUNCIONARIO']].sort_values(by=['DATA','HORAS'], ascending= False)
display(comp)

,DATA,HORAS,COD,DESCRIÇÃO,MOTIVO,FUNCIONARIO
14056,2025-09-02,05:05:00,428511,ESPONJA 3M LIMPEZA SC BRITE NAO RISCA,QUANTIDADE INFERIOR DA OS,ALEANDRA TRINDADE SOUSA
14040,2025-09-01,11:48:00,428511,ESPONJA 3M LIMPEZA SC BRITE NAO RISCA,QUANTIDADE INFERIOR DA OS,CARLOS ALBERTO SANTOS DA SILVA FILHO
